# Structured Output Parsers trong LangChain

## Giới thiệu

Structured Output Parsers giúp chúng ta:
- **Trích xuất dữ liệu có cấu trúc** từ text response của LLM
- **Đảm bảo type safety** với Pydantic models
- **Validate dữ liệu** tự động
- **Tạo schema phức tạp** cho các use case thực tế

Trong notebook này, chúng ta sẽ tập trung vào:
1. **PydanticOutputParser**: Parser mạnh mẽ nhất cho structured data
2. **JsonOutputParser**: Parser đơn giản cho JSON output

## Setup môi trường

In [ ]:
# Import các thư viện cần thiết
from langchain_core.output_parsers import PydanticOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_anthropic import ChatAnthropic
from pydantic import BaseModel, Field, validator
from typing import List, Optional, Dict
from datetime import datetime
import json
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Khởi tạo model
model = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    temperature=0,
    anthropic_api_key=os.getenv("ANTHROPIC_API_KEY")
)

## 1. PydanticOutputParser - Cơ bản

PydanticOutputParser sử dụng Pydantic models để định nghĩa schema cho output.

In [ ]:
# Định nghĩa Pydantic model đơn giản
class Person(BaseModel):
    name: str = Field(description="Tên đầy đủ của người")
    age: int = Field(description="Tuổi của người")
    occupation: str = Field(description="Nghề nghiệp hiện tại")
    city: str = Field(description="Thành phố đang sống")

# Tạo parser từ Pydantic model
person_parser = PydanticOutputParser(pydantic_object=Person)

# Xem format instructions
print("Format Instructions:")
print(person_parser.get_format_instructions())

In [ ]:
# Tạo prompt với format instructions
person_prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là trợ lý AI giúp trích xuất thông tin về người."),
    ("human", """Từ đoạn văn sau, hãy trích xuất thông tin về người được nhắc đến:
    
    {text}
    
    {format_instructions}""")
])

# Tạo chain
person_chain = person_prompt | model | person_parser

# Test với văn bản
test_text = """Nguyễn Văn A là một kỹ sư phần mềm 28 tuổi đang làm việc tại Hà Nội. 
Anh ấy đã có 5 năm kinh nghiệm trong ngành công nghệ."""

result = person_chain.invoke({
    "text": test_text,
    "format_instructions": person_parser.get_format_instructions()
})

print("Kết quả parsing:")
print(f"Type: {type(result)}")
print(f"Name: {result.name}")
print(f"Age: {result.age}")
print(f"Occupation: {result.occupation}")
print(f"City: {result.city}")

## 2. Pydantic Model phức tạp với nested structures

In [ ]:
# Định nghĩa models phức tạp hơn
class Address(BaseModel):
    street: str = Field(description="Tên đường")
    district: str = Field(description="Quận/Huyện")
    city: str = Field(description="Thành phố")
    
class Company(BaseModel):
    name: str = Field(description="Tên công ty")
    industry: str = Field(description="Ngành nghề kinh doanh")
    employee_count: Optional[int] = Field(description="Số lượng nhân viên", default=None)
    address: Address = Field(description="Địa chỉ công ty")
    
class Employee(BaseModel):
    name: str = Field(description="Họ tên nhân viên")
    position: str = Field(description="Vị trí/chức vụ")
    years_experience: int = Field(description="Số năm kinh nghiệm")
    skills: List[str] = Field(description="Danh sách kỹ năng")
    company: Company = Field(description="Thông tin công ty")
    
    # Validator để đảm bảo skills không empty
    @validator('skills')
    def skills_not_empty(cls, v):
        if not v:
            raise ValueError('Skills list cannot be empty')
        return v

In [ ]:
# Tạo parser và prompt cho Employee
employee_parser = PydanticOutputParser(pydantic_object=Employee)

employee_prompt = ChatPromptTemplate.from_messages([
    ("system", """Bạn là chuyên gia phân tích hồ sơ nhân sự.
    Hãy trích xuất thông tin chi tiết về nhân viên và công ty từ văn bản."""),
    ("human", """Phân tích thông tin sau:
    
    {text}
    
    {format_instructions}""")
])

# Test với văn bản phức tạp
complex_text = """Trần Thị B là một Senior Developer tại FPT Software, công ty công nghệ 
hàng đầu Việt Nam với hơn 20,000 nhân viên. Văn phòng chính của công ty tọa lạc tại 
đường Duy Tân, quận Cầu Giấy, Hà Nội. Với 7 năm kinh nghiệm, chị B thành thạo nhiều 
công nghệ như Java, Python, React và có khả năng quản lý dự án tốt."""

# Tạo chain và parse
employee_chain = employee_prompt | model | employee_parser

result = employee_chain.invoke({
    "text": complex_text,
    "format_instructions": employee_parser.get_format_instructions()
})

print("=== Thông tin nhân viên ===")
print(f"Tên: {result.name}")
print(f"Vị trí: {result.position}")
print(f"Kinh nghiệm: {result.years_experience} năm")
print(f"Kỹ năng: {', '.join(result.skills)}")
print("\n=== Thông tin công ty ===")
print(f"Tên công ty: {result.company.name}")
print(f"Ngành: {result.company.industry}")
print(f"Số nhân viên: {result.company.employee_count}")
print(f"Địa chỉ: {result.company.address.street}, {result.company.address.district}, {result.company.address.city}")

## 3. JsonOutputParser - Đơn giản và linh hoạt

In [ ]:
# JsonOutputParser không cần schema cứng
json_parser = JsonOutputParser()

# Prompt yêu cầu JSON output
json_prompt = ChatPromptTemplate.from_messages([
    ("system", "Bạn là trợ lý phân tích sản phẩm."),
    ("human", """Phân tích sản phẩm sau và trả về JSON với các trường:
    - name: tên sản phẩm
    - category: danh mục
    - price: giá (số)
    - features: danh sách tính năng (array)
    - rating: đánh giá từ 1-5
    
    Sản phẩm: {product_description}
    
    {format_instructions}""")
])

# Test
product_desc = """iPhone 15 Pro Max - điện thoại cao cấp với chip A17 Pro, 
camera 48MP, màn hình 6.7 inch, pin dùng cả ngày. Giá 35 triệu đồng. 
Người dùng đánh giá 4.5/5 sao."""

json_chain = json_prompt | model | json_parser

result = json_chain.invoke({
    "product_description": product_desc,
    "format_instructions": json_parser.get_format_instructions()
})

print("Kết quả (type):", type(result))
print("\nJSON output:")
print(json.dumps(result, indent=2, ensure_ascii=False))

## 4. Use Case: Trích xuất thông tin từ CV

In [ ]:
# Model cho CV parsing
class Education(BaseModel):
    degree: str = Field(description="Bằng cấp (Cử nhân, Thạc sĩ, ...)")
    major: str = Field(description="Chuyên ngành")
    university: str = Field(description="Tên trường")
    graduation_year: Optional[int] = Field(description="Năm tốt nghiệp")

class WorkExperience(BaseModel):
    company: str = Field(description="Tên công ty")
    position: str = Field(description="Vị trí/chức vụ")
    duration: str = Field(description="Thời gian làm việc")
    responsibilities: List[str] = Field(description="Các trách nhiệm chính")

class CVInfo(BaseModel):
    full_name: str = Field(description="Họ tên đầy đủ")
    email: Optional[str] = Field(description="Email liên hệ")
    phone: Optional[str] = Field(description="Số điện thoại")
    summary: str = Field(description="Tóm tắt về ứng viên")
    skills: List[str] = Field(description="Danh sách kỹ năng")
    education: List[Education] = Field(description="Học vấn")
    experience: List[WorkExperience] = Field(description="Kinh nghiệm làm việc")
    
# Parser và prompt
cv_parser = PydanticOutputParser(pydantic_object=CVInfo)

cv_prompt = ChatPromptTemplate.from_messages([
    ("system", """Bạn là chuyên gia phân tích CV. 
    Hãy trích xuất thông tin chi tiết và chính xác từ CV."""),
    ("human", """Phân tích CV sau:
    
    {cv_text}
    
    {format_instructions}""")
])

In [ ]:
# Test với CV mẫu
sample_cv = """NGUYỄN VĂN MINH
Email: minh.nguyen@email.com | Phone: 0901234567

GIỚI THIỆU
Kỹ sư phần mềm với 5 năm kinh nghiệm phát triển web applications. 
Chuyên môn về Python, Django, React với đam mê xây dựng sản phẩm chất lượng cao.

HỌC VẤN
- Cử nhân Công nghệ Thông tin, Đại học Bách Khoa Hà Nội (2018)
  Chuyên ngành: Kỹ thuật phần mềm

KINH NGHIỆM LÀM VIỆC

1. Senior Developer - VNG Corporation (2021 - Hiện tại)
   - Phát triển và maintain hệ thống payment gateway phục vụ 10M+ users
   - Lead team 5 developers, review code và mentor junior members
   - Tối ưu performance, giảm 40% response time của API

2. Full-stack Developer - Tiki.vn (2019 - 2021)
   - Xây dựng tính năng recommendation engine cho e-commerce platform
   - Phát triển admin dashboard sử dụng React và Material-UI
   - Tham gia migration từ monolith sang microservices

KỸ NĂNG
- Ngôn ngữ: Python, JavaScript, Go
- Framework: Django, FastAPI, React, Next.js
- Database: PostgreSQL, MongoDB, Redis
- DevOps: Docker, Kubernetes, AWS"""

# Parse CV
cv_chain = cv_prompt | model | cv_parser

cv_result = cv_chain.invoke({
    "cv_text": sample_cv,
    "format_instructions": cv_parser.get_format_instructions()
})

# In kết quả
print("=== THÔNG TIN ỨNG VIÊN ===")
print(f"Họ tên: {cv_result.full_name}")
print(f"Email: {cv_result.email}")
print(f"SĐT: {cv_result.phone}")
print(f"\nTóm tắt: {cv_result.summary}")

print("\n=== HỌC VẤN ===")
for edu in cv_result.education:
    print(f"- {edu.degree} {edu.major} - {edu.university} ({edu.graduation_year})")

print("\n=== KINH NGHIỆM ===")
for exp in cv_result.experience:
    print(f"\n{exp.position} - {exp.company} ({exp.duration})")
    for resp in exp.responsibilities:
        print(f"  • {resp}")

print("\n=== KỸ NĂNG ===")
print(", ".join(cv_result.skills))

## 5. Error Handling và Retry Logic

In [ ]:
from langchain_core.output_parsers import OutputFixingParser
from langchain_core.exceptions import OutputParserException

# Model với validation phức tạp
class Product(BaseModel):
    name: str = Field(description="Tên sản phẩm")
    price: float = Field(description="Giá sản phẩm", gt=0)  # phải > 0
    stock: int = Field(description="Số lượng tồn kho", ge=0)  # phải >= 0
    
    @validator('name')
    def name_not_empty(cls, v):
        if not v or not v.strip():
            raise ValueError('Product name cannot be empty')
        return v.strip()

# Tạo parser với error fixing
base_parser = PydanticOutputParser(pydantic_object=Product)
fixing_parser = OutputFixingParser.from_llm(parser=base_parser, llm=model)

# Test với input có thể gây lỗi
test_prompt = ChatPromptTemplate.from_messages([
    ("human", """Tạo thông tin sản phẩm từ mô tả sau:
    {description}
    
    {format_instructions}""")
])

# Test cases
test_cases = [
    "Laptop Dell XPS 13 giá 25 triệu, còn 5 chiếc",
    "Sản phẩm hết hàng, giá đang khuyến mãi",  # Có thể gây lỗi
    "Tai nghe bluetooth giá âm 100k"  # Sẽ gây lỗi validation
]

for desc in test_cases:
    print(f"\nTest: {desc}")
    print("-" * 50)
    
    try:
        # Thử với base parser
        chain = test_prompt | model | base_parser
        result = chain.invoke({
            "description": desc,
            "format_instructions": base_parser.get_format_instructions()
        })
        print(f"✓ Base parser success: {result}")
        
    except Exception as e:
        print(f"✗ Base parser failed: {str(e)}")
        
        # Thử với fixing parser
        try:
            fixing_chain = test_prompt | model | fixing_parser
            result = fixing_chain.invoke({
                "description": desc,
                "format_instructions": base_parser.get_format_instructions()
            })
            print(f"✓ Fixing parser success: {result}")
        except Exception as e2:
            print(f"✗ Fixing parser also failed: {str(e2)}")

## 6. Custom Output Parser

In [ ]:
from langchain_core.output_parsers import BaseOutputParser
from typing import Type, TypeVar

T = TypeVar('T', bound=BaseModel)

class RobustPydanticOutputParser(BaseOutputParser[T]):
    """Custom parser với multiple retry strategies"""
    
    pydantic_object: Type[T]
    
    def parse(self, text: str) -> T:
        """Parse với multiple strategies"""
        # Strategy 1: Try direct JSON parsing
        try:
            # Tìm JSON trong text
            import re
            json_match = re.search(r'\{[^{}]*\}', text, re.DOTALL)
            if json_match:
                json_str = json_match.group()
                data = json.loads(json_str)
                return self.pydantic_object(**data)
        except:
            pass
        
        # Strategy 2: Try parsing entire text as JSON
        try:
            data = json.loads(text)
            return self.pydantic_object(**data)
        except:
            pass
        
        # Strategy 3: Extract key-value pairs manually
        try:
            # Simple extraction logic
            lines = text.strip().split('\n')
            data = {}
            for line in lines:
                if ':' in line:
                    key, value = line.split(':', 1)
                    key = key.strip().lower().replace(' ', '_')
                    value = value.strip()
                    # Try to convert to appropriate type
                    try:
                        value = json.loads(value)
                    except:
                        pass
                    data[key] = value
            return self.pydantic_object(**data)
        except Exception as e:
            raise OutputParserException(f"Failed to parse output: {str(e)}")
    
    def get_format_instructions(self) -> str:
        schema = self.pydantic_object.schema()
        return f"Please output a valid JSON object that conforms to this schema:\n{json.dumps(schema, indent=2)}"

# Test custom parser
class SimpleProduct(BaseModel):
    name: str
    price: float

custom_parser = RobustPydanticOutputParser(pydantic_object=SimpleProduct)

# Test với các format khác nhau
test_outputs = [
    '{"name": "iPhone 15", "price": 999}',  # Valid JSON
    'name: iPhone 15\nprice: 999',  # Key-value format
    'The product is iPhone 15 with price $999. {"name": "iPhone 15", "price": 999}',  # Mixed
]

for output in test_outputs:
    try:
        result = custom_parser.parse(output)
        print(f"✓ Parsed successfully: {result}")
    except Exception as e:
        print(f"✗ Failed to parse: {str(e)}")

## 7. Best Practices cho Structured Output

In [ ]:
# Best Practice 1: Clear field descriptions
class WellDocumentedModel(BaseModel):
    """Model với documentation rõ ràng"""
    
    product_id: str = Field(
        description="Mã sản phẩm duy nhất, format: PRD-XXXXX",
        regex="^PRD-[0-9]{5}$"
    )
    
    price_vnd: float = Field(
        description="Giá bằng VND, phải là số dương",
        gt=0,
        le=1_000_000_000  # Max 1 tỷ
    )
    
    categories: List[str] = Field(
        description="Danh mục sản phẩm, tối thiểu 1, tối đa 3",
        min_items=1,
        max_items=3
    )
    
    available: bool = Field(
        description="True nếu còn hàng, False nếu hết",
        default=True
    )

# Best Practice 2: Examples in prompt
example_prompt = ChatPromptTemplate.from_messages([
    ("system", """Bạn là trợ lý phân tích sản phẩm.
    
    Ví dụ output mong muốn:
    {{
        "product_id": "PRD-12345",
        "price_vnd": 2500000,
        "categories": ["Điện thoại", "Apple"],
        "available": true
    }}"""),
    ("human", """Phân tích: {description}
    
    {format_instructions}""")
])

# Best Practice 3: Fallback values
class SafeModel(BaseModel):
    name: str
    quantity: int = Field(default=0, description="Số lượng, mặc định 0")
    tags: List[str] = Field(default_factory=list, description="Tags, có thể rỗng")
    metadata: Dict[str, any] = Field(default_factory=dict)

print("Well-documented schema:")
print(json.dumps(WellDocumentedModel.schema(), indent=2, ensure_ascii=False))

## 8. Production Example: API Response Parser

In [ ]:
# Production-ready parser cho API responses
from enum import Enum
from datetime import datetime

class OrderStatus(str, Enum):
    PENDING = "pending"
    CONFIRMED = "confirmed"
    SHIPPING = "shipping"
    DELIVERED = "delivered"
    CANCELLED = "cancelled"

class OrderItem(BaseModel):
    product_name: str
    quantity: int = Field(gt=0)
    unit_price: float = Field(gt=0)
    
    @property
    def total_price(self) -> float:
        return self.quantity * self.unit_price

class Order(BaseModel):
    order_id: str = Field(description="Mã đơn hàng")
    customer_name: str
    customer_phone: str = Field(regex="^[0-9]{10,11}$")
    items: List[OrderItem] = Field(min_items=1)
    status: OrderStatus
    notes: Optional[str] = None
    created_at: datetime = Field(default_factory=datetime.now)
    
    @property
    def total_amount(self) -> float:
        return sum(item.total_price for item in self.items)
    
    class Config:
        use_enum_values = True

# Parser và chain
order_parser = PydanticOutputParser(pydantic_object=Order)

order_prompt = ChatPromptTemplate.from_messages([
    ("system", """Extract order information from customer messages.
    Ensure all required fields are populated correctly."""),
    ("human", """Customer message: {message}
    
    {format_instructions}""")
])

# Test với tin nhắn khách hàng
customer_message = """Chào shop, tôi muốn đặt:
- 2 áo thun trắng size L giá 150k/cái  
- 1 quần jean xanh size 32 giá 450k
Tên tôi là Phạm Văn Hùng, SĐT 0912345678.
Giao hàng nhanh giúp tôi nhé, tôi cần trước thứ 6."""

order_chain = order_prompt | model | order_parser

try:
    order = order_chain.invoke({
        "message": customer_message,
        "format_instructions": order_parser.get_format_instructions()
    })
    
    print("=== ĐƠN HÀNG ===")
    print(f"Mã đơn: {order.order_id}")
    print(f"Khách hàng: {order.customer_name} - {order.customer_phone}")
    print(f"Trạng thái: {order.status}")
    print(f"\nSản phẩm:")
    for i, item in enumerate(order.items, 1):
        print(f"  {i}. {item.product_name} x{item.quantity} = {item.total_price:,.0f}đ")
    print(f"\nTổng tiền: {order.total_amount:,.0f}đ")
    if order.notes:
        print(f"Ghi chú: {order.notes}")
        
except Exception as e:
    print(f"Error parsing order: {str(e)}")

## Tổng kết

Trong notebook này, chúng ta đã học về Structured Output Parsers:

1. **PydanticOutputParser**:
   - Type-safe với Pydantic models
   - Validation tự động
   - Nested structures support

2. **JsonOutputParser**:
   - Đơn giản, linh hoạt
   - Không cần schema cứng

3. **Advanced Features**:
   - Error handling với OutputFixingParser
   - Custom parsers cho special cases
   - Validators và business logic

4. **Best Practices**:
   - Clear field descriptions
   - Examples trong prompts
   - Fallback values
   - Production-ready patterns

Structured Output Parsers là công cụ essential để build production applications với LLMs!